# 1. Preparation
#### First we install the required packages

In [ ]:
install.packages(c('network'), .libPaths(), repos='http://cran.us.r-project.org', type = 'binary')
install.packages(c('ergm'), .libPaths(), repos='http://cran.us.r-project.org', type = 'binary')
install.packages(c('devtools'), .libPaths(), repos='http://cran.us.r-project.org', type = 'binary')

#### This includes the workshop package that contains the data we will be working with and some utility functions.

In [ ]:
devtools::install_github('tedhchen/ergmWorkshop', force = T)

#### Next, we load the libraries.

In [ ]:
library(ergm)
library(ergmWorkshop)

As you can see from the output, loading the `ergm` package also loaded the `network` package.

There are also these references to the `statnet` project. They are the main developers of ERGM functionalities in `R`.

# 2. Building our networks
#### First, load the data.
The data sets we will be using are part of the workshop package. We start with the militarized interstate dispute data from Zeev Maoz.

In [ ]:
data(mid_mat)
?mid_mat

In [ ]:
nw <- network(mid_mat, directed = F)
nw

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)
par(mfrow = c(1, 2))
set.seed(210615); plot(nw)

#### There are different ways network data is usually stored.
The adjacency matrix format is generally the easiest to work with, but it can waste a lot of space.

Sometimes, we will have edgelists instead. But we need to be careful when working with them.

In [ ]:
data(mid_edgelist)
nw.b <- network(mid_edgelist, directed = F)
nw.b

#### What's the difference?

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)
par(mfrow = c(1, 2))
set.seed(210615); plot(nw); plot(nw.b)

#### Where are the isolates?

In [ ]:
data(mid_node_attr)
head(mid_node_attr)
?mid_node_attr

#### Let's manually make the adjacency matrix from the edgelist.
- Start by making an empty matrix.

In [ ]:
mat <- matrix(0, ncol = 189, nrow = 189, dimnames = list(row.names(mid_node_attr), row.names(mid_node_attr)))
mat[1:10, 1:10]

- Next, loop through the edges and fill the correct cells.

In [ ]:
for(i in 1:nrow(mid_edgelist)){
    mat[mid_edgelist[i, 1], mid_edgelist[i, 2]] <- 1
    mat[mid_edgelist[i, 2], mid_edgelist[i, 1]] <- 1
}

`checkmat()` is a utility function that plots the supplied adjacency matrix.

In [ ]:
options(repr.plot.width=14, repr.plot.height=7)
par(mfrow = c(1, 2))
checkmat(mat);checkmat(mid_mat)

#### Let's get back to working with our network.

In [ ]:
nw

#### Let's work with vertex attributes

In [ ]:
nw%v%'vertex.names'

In [ ]:
nw%v%'dem' <- mid_node_attr$'dem'
nw

In [ ]:
nw%v%'dem'

#### Red nodes are democracies and black are nondemocracies

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
set.seed(210615); plot(nw, vertex.col = ifelse(nw%v%'dem', 2, 1)); legend('bottomright', legend = c('Democracy', 'Nondemocracy'), fill = c(2, 1), bty = 'n', cex = 1.1)

#### Edge attributes can also be specified in a similar way but it's easier to just use a separate matrix
These are the geographical contiguity and joint-democracy networks.

In [ ]:
data(contig); data(joint_dem)

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)
par(mfrow = c(1, 2))
set.seed(210615); plot(network(contig, directed = F), sub = 'Contiguity', cex.sub = 2); plot(network(joint_dem, directed = F), sub = 'Joint-Democracy', cex.sub = 2)

# 3. Specifying the ERGM

#### Let's take a look at the basic function for ERGM fitting: `ergm`

`ergm(network ~ ergm-terms)`

#### What are the ERGM terms that we can use?

## 3.1 ERGM Terms
- use ``?`ergm-terms'`` to look at all the existing terms in the `ergm` package (not demonstrating because it does not render well in a notebook)
- the `search.ergmTerms` function also works well

In [ ]:
search.ergmTerms(keyword = 'transitive')

## 3.2 Starting to fit an ERGM
#### We start with some node and dyad variables

In [ ]:
m0 <- ergm(nw ~ edges + nodefactor('dem') + edgecov(joint_dem))
summary(m0)

#### Let's compare it with the logistic regression

In [ ]:
dyad_df <- ergmMPLE(nw ~ edges + nodefactor('dem') + edgecov(joint_dem))
dyad_df$predictor

In [ ]:
summary(glm(dyad_df$response ~ dyad_df$predictor - 1, weights = dyad_df$weights, family = 'binomial'))

#### Let's move to some network effects
- start with the "pile-on" (or popularity) effect
-`kstar(2)` (which is the 2-star term) is the basic term

In [ ]:
m1 <- ergm(nw ~ edges + nodefactor('dem') + edgecov(joint_dem) + kstar(2))

#### What happened here?
- This is one of the difficulties of ERGMs: they run into degeneracy issues.
- Degeneracy means...

So instead, we can use alternative network statistics designed to address these problems.
- For the popularity (or pile-on) effect, the commonly-used one is the geometrically-weighted degree.
- In the `ergm` package, it is called `gwdegree`.

References:


In [ ]:
m1 <- ergm(nw ~ edges + nodefactor('dem') + edgecov(joint_dem) + gwdegree(decay = 0, fixed = T))

#### Looks like it converged

In [ ]:
summary(m1)

- the `gwdeg.fixed.0` term has coefficient of -1.17
- **for the geometrically weighted degree term, negative means popular**
    - This is an easy mistake: a review in 2016 shows that only 4/16 papers got it right.
- compare the model fit statistics: AIC and BIC are both much better than without fitting the pile-on effect

#### Let's also add the triadic closure effect before looking at the model in more detail

In [ ]:
m2 <- ergm(nw ~ edges + nodefactor('dem') + edgecov(joint_dem) + gwdegree(decay = 0, fixed = T) + triangle)

#### Again, seems like we have degeneracy
- we can use the geometrically weighted edgewise shared partners statistic
- in the `ergm` package, it is called `gwesp`

In [ ]:
m2 <- ergm(nw ~ edges + nodefactor('dem') + edgecov(joint_dem) + gwdegree(decay = 0, fixed = T) + gwesp(decay = 0, fixed = T))

In [ ]:
summary(m2)

#### Looks like it converged
- `gwesp.fixed.0` is positive here
- this is against what we expected, but it's hard to draw conclusions since the model is quite rudimentary
- model fit is improved again; this is generally what happens with the `gwdegree` and `gwesp` terms

### 3.2.1 Let's consider model fit, degeneracy, and decay parameters
#### How to understand the decay parameters?
- the lower the decay value, the less likely you are going to run into degeneracy problems
- at the same time, a lower decay value also makes the model fit worse because you are removing information (generally speaking)
- for the geometrically weighted degree, lower decay values makes additional clustered ties matter less (the same interpretation works for additional edgewise shared partners in the `gwesp`)
- the model is less able to discriminate between the network effect of small and large clusters
- The most useful tool when working with geometrically weighted degrees: https://github.com/michaellevy/gwdegree

#### This is easiest to demonstrate using the `gwesp`
- let's make a fully saturated direct network with 10 nodes
- this network will have $10\times 9 = 90$ edges
- every edge will have 8 edgewise shared partners

In [ ]:
tenclique <- network(matrix(1, ncol = 10, nrow = 10), directed = T)
options(repr.plot.width=7, repr.plot.height=7)
set.seed(210615); plot(tenclique)

In [ ]:
summary(tenclique ~ edges + gwesp(0, fixed = T) + gwesp(0.5, fixed = T) + gwesp(1, fixed = T) + gwesp(2, fixed = T) + gwesp(5, fixed = T) + gwesp(10, fixed = T) + ttriple)

#### What happened?
- `gwesp` with decay set to 0 means every edge has "one or more" shared partner - this makes it the same value as the `edge` count term
- as we increase the decay value, it starts to converge toward the `ttriple` term, which is a count of all edgewise shared partners of all edges (i.e. $10\times 9\times 8 = 720$)
- on the one end, you will have an unidentified model (i.e. `edges` and `gwesp` are the same) and on the other end, you will run into degeneracy issues



#### Let's look at a real example using our MIDs network
- let's increase the decay value to 0.2

In [ ]:
m2.b <- ergm(nw ~ edges + nodefactor('dem') + edgecov(joint_dem) + gwdegree(decay = 0.2, fixed = T) + gwesp(decay = 0, fixed = T))

In [ ]:
summary(m2.b)

#### Slightly increasing the decay value improved fit and didn't run into degeneracy issues

## 3.3 Assessing ERGM fit
#### Back to our MIDs model; it seems like our model converged, but we should still check the fit using simulations
- the ERGM is a generative model, so it can be used to simulate networks that have the same generative features (parameters and configurations)
- simulate a bunch of networks, which lets us check how well our model captures the observed network
- the `gof` function is what we use

In [ ]:
m2fit <- gof(m2)
m2fit

#### What are we looking at?
- we simulated a bunch of networks, and computed some important network statistics
- the output above are summaries of the simulations compared to the observed value
- it's easier to see when we plot them

In [ ]:
options(repr.plot.width=32, repr.plot.height=8)
par(mfrow = c(1, 4))
plot(m2fit)

- black line shows the observed values
- the box and whisker plots show the distributions of the simulated networks
- ideally, we want the observed values to fall in the gray boxes
- we also don't want to "overfit"
- there isn't an exact answer to this task

#### But it helps to compare

In [ ]:
m1fit <- gof(m1)
options(repr.plot.width=32, repr.plot.height=8)
par(mfrow = c(1, 4))
plot(m1fit)

## 3.4 Interpreting the ERGM output
#### Let's bring up the model output again

In [ ]:
summary(m2)

#### At the network level:
- positive coefficient means positive contribution to the network generative process
- for example, positive `gwesp` means triadic closure is a feature of the network

#### At the dyad level:
- how many configurations are each edge a part of? (this is the change statistic)
- `nodefactor.dem` will be 0 if two nondemocracies, 1 if mixed regime, and 2 if joint democracies
- `gwesp.fixed.0` will be 1 if the dyad is part of at least one triangle

#### $P(y_{ij} = 1 | Y, \mathbf{\theta})  = logit ^{-1}(\sum^k_{r=1}\theta_r \delta_r^{(ij)}Y)$
- conditional log odds of a tie given a one unit change in the statistic
- for the less complicated terms, it becomes the conditional log odds of a tie given that it is part of the structure one time

#### Let's illustrate using a simple example and the same coefficients
Just a simple three node network with two ties to show tie formation on the empty dyad

In [ ]:
tri <- matrix(c(0, 1, 1,
                1, 0, 0,
                1, 0, 0), byrow = T, ncol = 3)
nw.tri <- network(tri, directed = F)
nw.tri%v%'dem' <- c(0, 1, 1)
jd.tri <- matrix(c(0, 0, 0,
                   0, 0, 1,
                   0, 1, 0), byrow = T, ncol = 3)
options(repr.plot.width=10, repr.plot.height=10)
set.seed(210615); plot(nw.tri, vertex.col = ifelse(nw.tri%v%'dem', 2, 1), displaylabels = T)

#### What's the probability of a tie forming on the empty dyad if this network has the same generative features of our conflict network?
- we need to see how the addition of the tie will change the network statistics
- the `ergmMPLE` function can help with this, especially with more complicated terms like the geometrically weighted ones

In [ ]:
ergmMPLE(nw.tri ~ edges + nodefactor('dem') + edgecov(jd.tri) + gwdegree(decay = 0, fixed = T) + gwesp(decay = 0, fixed = T))$predictor

#### Adding an edge on the 2-3 dyad will add...
- 1 edge
- 2 democracies
- 1 joint democracy dyad
- 0 to the `gwdegree.0` term
- 3 to the `gwesp.0` term

#### Why does `gwdegree` not change?
Let's look at the degree distribution:
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg" align = "left">
<thead>
  <tr>
    <th class="tg-0pky">nodes with degree:</th>
    <th class="tg-0pky">0</th>
    <th class="tg-0pky">1</th>
    <th class="tg-0pky">2</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">w/o edge</td>
    <td class="tg-0pky">0</td>
    <td class="tg-0pky">2</td>
    <td class="tg-0pky">1</td>
  </tr>
  <tr>
    <td class="tg-0pky">w/ edge</td>
    <td class="tg-0pky">0</td>
    <td class="tg-0pky">0</td>
    <td class="tg-0pky">3</td>
  </tr>
</tbody>
</table>

- When the decay value is 0, all node degrees are treated the same
- Adding the (2, 3) edge removes two degree-1 nodes, and adds two degree-2 nodes so `gwdegree.0` does not change

#### Calculate the log-odds
$(-4.70 \times 1) + (-0.13 \times 2) + (-0.25 \times 1) + (-0.64 \times 0) + (1.14 \times 3) = -1.79$

#### Calculate the corresponding probability

In [ ]:
plogis(-1.79)

- generally speaking, it will get confusing to calculate the probability of a tie forming as the networks get more complex and the model terms more nested

#### At the block level
- Using simulations to look at changes
- Beyond the scope of what we can do in this tutorial
- See the paper in PSJ

Reference:

## 3.5 ERGM Settings

#### Let's conclude this section by looking at some settings and useful things to do
- turn off `eval.loglik` if you are exploring large and complex models
- set a seed to ensure consistency
- increase `MCMC.burnin`, `MCMC.samplesize`, and `MCMC.interval` for higher quality estimates
- use `parallel` to speed up estimation when working with larger models

In [ ]:
m2.c <- ergm(nw ~ edges + nodefactor('dem') + edgecov(joint_dem) + gwdegree(decay = 0, fixed = T) + gwesp(decay = 0, fixed = T),
             eval.loglik = F,
             control = control.ergm(seed = 111953,
                                    MCMC.burnin = 5000,
                                    MCMC.samplesize = 2000,
                                    MCMC.interval = 2000,
                                    parallel = 0))

In [ ]:
summary(m2.c)

#### We don't get any likelihood-based statistics. We can follow the instructions to add them.

In [ ]:
m2.c <- logLik(m2.c, add = T)

In [ ]:
summary(m2.c)

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)
mcmc.diagnostics(m2)

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)
mcmc.diagnostics(m2.c)

# 4. References
